In [1]:
import jpype
import jpype.imports
from jpype.types import *
from zemberek.morphology import TurkishMorphology
import speech_recognition as sr
import pandas as pd
import string

ZEMBEREK_PATH = "zemberek-full.jar"

if not jpype.isJVMStarted():
    jpype.startJVM(classpath=[ZEMBEREK_PATH])


In [4]:
morphology = TurkishMorphology.create_with_defaults()

context = """Yangın, Hançerli Mahallesi'ndeki zeytinlikte öğle saatlerinde çıktı. 
Alevlerin yükseldiğini görenlerin ihbarıyla bölgeye itfaiye ve jandarma ekipleri sevk edildi. 
Alevler, rüzgarın etkisiyle ormana sıçradı. Yangın, itfaiye ekiplerinin müdahalesi ile kontrol altına alındı. 
Bölgede 5 araç ve 16 personel ile soğutma çalışmaları sürerken, yangının çıkış nedeniyle ilgili inceleme başlatıldı."
analysis = morphology.analyze(word)"""

words = context.split()

for word in words:
    word = word.strip(string.punctuation)
    if word:
        try:
            analysis = morphology.analyze(word)
            for result in analysis:
                print(result)
        except Exception as e:
            print("Error")
# jpype.shutdownJVM()


2024-10-06 21:56:40,251 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 15.579967975616455

[yangın:Adj] yangın:Adj
[Yang:Noun, Prop] yang:Noun+A3sg+ın:Gen
[Yang:Noun, Prop] yang:Noun+A3sg+ın:P2sg
[yangı:Noun] yangı:Noun+A3sg+n:P2sg
[yangın:Noun] yangın:Noun+A3sg
[hançer:Noun] hançer:Noun+A3sg|li:With→Adj
[mahalle:Noun] mahalle:Noun+A3sg+si:P3sg+nde:Loc|ki:Rel→Adj
[Mahallesi:Noun, Prop] mahallesi:Noun+A3sg+n:P2sg+de:Loc|ki:Rel→Adj
[zeytinlik:Noun] zeytinlik:Noun+A3sg+te:Loc
[zeytin:Adj] zeytin:Adj|lik:Ness→Noun+A3sg+te:Loc
[zeytin:Noun] zeytin:Noun+A3sg|lik:Ness→Noun+A3sg+te:Loc
[öğle:Noun] öğle:Noun+A3sg
[saat:Noun] saat:Noun+A3sg+leri:P3pl+nde:Loc
[saat:Noun] saat:Noun+ler:A3pl+in:P2sg+de:Loc
[saat:Noun] saat:Noun+ler:A3pl+i:P3sg+nde:Loc
[saat:Noun] saat:Noun+ler:A3pl+i:P3pl+nde:Loc
[çıkmak:Verb] çık:Verb+tı:Past+A3sg
[çıktı:Noun] çıktı:Noun+A3sg
[Alev:Noun, Prop] alev:Noun+ler:A3pl+in:Gen
[Alev:Noun, Prop] alev:Noun+ler:A3pl+in:P2sg
[al

In [15]:
recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("Dinleniyor:")
    audio = recognizer.listen(source)

try:
    stt_text = recognizer.recognize_google(audio, language="tr-TR")  
    print(f"Recognized Text: {stt_text}")
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand the audio")
except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")
    

Dinleniyor:
Recognized Text: sadeleştirilmiş mi


In [16]:
if stt_text != " ":
    stt_words = stt_text.split()

    for stt_word in stt_words:
        stt_analysis = morphology.analyze(stt_word)
        for stt_result in stt_analysis:
            print(stt_result)

[sade:Adj] sade:Adj|leş:Become→Verb|tir:Caus→Verb|il:Pass→Verb|miş:NarrPart→Adj
[sade:Adj] sade:Adj|leş:Become→Verb|tir:Caus→Verb|il:Pass→Verb+miş:Narr+A3sg
[Sade:Noun, Prop] sade:Noun+A3sg|leş:Become→Verb|tir:Caus→Verb|il:Pass→Verb|miş:NarrPart→Adj
[Sade:Noun, Prop] sade:Noun+A3sg|leş:Become→Verb|tir:Caus→Verb|il:Pass→Verb+miş:Narr+A3sg
[mi:Ques] mi:Ques+Pres+A3sg
[Mi:Noun, Abbrv] mi:Noun+A3sg
[mi:Noun] mi:Noun+A3sg


In [17]:
file_path = 'train.tsv'  # cv-corpus-16.1-2023-12-06
df = pd.read_csv(file_path, sep='\t')

print(df.head())

                                           client_id  \
0  f4347e919b8eef3cff43d962ea3ce1d06dcd8799acae73...   
1  f4347e919b8eef3cff43d962ea3ce1d06dcd8799acae73...   
2  f4347e919b8eef3cff43d962ea3ce1d06dcd8799acae73...   
3  f4347e919b8eef3cff43d962ea3ce1d06dcd8799acae73...   
4  f4347e919b8eef3cff43d962ea3ce1d06dcd8799acae73...   

                           path  \
0  common_voice_tr_28651153.mp3   
1  common_voice_tr_28651158.mp3   
2  common_voice_tr_28651159.mp3   
3  common_voice_tr_28651160.mp3   
4  common_voice_tr_28651161.mp3   

                                            sentence  up_votes  down_votes  \
0  Sabahleyin karşı karşıya gelince dişi söylemek...         2           0   
1                                         Güle güle!         2           0   
2  Kafamda onun hakkında, biraz evvel beliren ışı...         2           0   
3                                       Ne dersiniz?         2           0   
4                                   Beklediğim sesi.         2

In [18]:
for i in range(len(df["sentence"])):
    index = df["sentence"][i].split()
    print(f"Sentence {i+1}: {index}")
    
    for j in index:
        j = j.strip(string.punctuation)
        cv_analysis = morphology.analyze(j)
        for cv_result in cv_analysis:
            print(cv_result)
    

Sentence 1: ['Sabahleyin', 'karşı', 'karşıya', 'gelince', 'dişi', 'söylemek', 'istediği', 'şeyleri', 'gözleriyle', 'anlatmak', 'istedi.']
[sabahleyin:Adv] sabahleyin:Adv
[sabahleyin:Noun, Time] sabahleyin:Noun+A3sg
[karşı:Postp, PCDat] karşı:Postp
[karşı:Adv] karşı:Adv
[karşı:Adj] karşı:Adj
[karşı:Noun] karşı:Noun+A3sg
[karşı:Noun] karşı:Noun+A3sg+Dat
[karşı:Noun] karşı:Noun+A3sg+ya:Dat
[karşı:Adj] karşı:Adj|Zero→Noun+A3sg+ya:Dat
[gelmek:Verb] gel:Verb|ince:When→Adv
[gelin:Noun] gelin:Noun+A3sg+ce:Equ
[dişi:Adj] dişi:Adj
[diş:Noun] diş:Noun+A3sg+i:Acc
[diş:Noun] diş:Noun+A3sg+i:P3sg
[dişi:Noun] dişi:Noun+A3sg
[söylemek:Verb] söyle:Verb|mek:Inf1→Noun+A3sg
[istemek:Verb] iste:Verb|diğ:PastPart→Adj+i:P3sg
[istemek:Verb] iste:Verb|diğ:PastPart→Noun+A3sg+i:Acc
[istemek:Verb] iste:Verb|diğ:PastPart→Noun+A3sg+i:P3sg
[şey:Noun] şey:Noun+A3sg+leri:P3pl
[şey:Noun] şey:Noun+ler:A3pl+i:Acc
[şey:Noun] şey:Noun+ler:A3pl+i:P3sg
[şey:Noun] şey:Noun+ler:A3pl+i:P3pl
[göz:Noun] göz:Noun+A3sg+leri:P3pl+yl

[ibaret:Adj] ibaret:Adj|Zero→Verb+Pres+A3sg+tir:Cop
[ibaret:Noun] ibaret:Noun+A3sg|Zero→Verb+Pres+A3sg+tir:Cop
[kendi:Pron, Reflex] kendi:Pron+A3sg+P3sg
[Ken:Noun, Abbrv] ken:Noun+A3sg|Zero→Verb+di:Past+A3sg
[içeri:Noun] içeri:Noun+A3sg+si:P3sg+nde:Loc
[içeri:Adj] içeri:Adj|Zero→Noun+A3sg+si:P3sg+nde:Loc
[fark:Noun] fark:Noun+A3sg|lı:With→Adj|lık:Ness→Noun+lar:A3pl
[fark:Noun] fark:Noun+A3sg|lı:With→Adj|lık:Ness→Noun+A3sg|Zero→Verb+Pres+lar:A3pl
[barınmak:Verb] barın:Verb|dır:Caus→Verb+ma:Neg+z:Aor+A3sg
[barınmak:Verb] barın:Verb|dır:Caus→Verb+ma:Neg|z:AorPart→Adj
Sentence 22: ['Harp', 'sonu', 'senelerinin', 'dizginsiz', 'coşkunluğu', 'burada', 'bütün', 'çıplaklığıyla', 'görülüyordu.']
[harp:Noun] harp:Noun+A3sg
[son:Noun] son:Noun+A3sg+u:Acc
[son:Noun] son:Noun+A3sg+u:P3sg
[son:Num] son:Num|Zero→Noun+A3sg+u:Acc
[son:Num] son:Num|Zero→Noun+A3sg+u:P3sg
[son:Adj] son:Adj|Zero→Noun+A3sg+u:Acc
[son:Adj] son:Adj|Zero→Noun+A3sg+u:P3sg
[sene:Noun, Time] sene:Noun+A3sg+leri:P3pl+nin:Gen
[sene:

[sicim:Noun] sicim:Noun+A3sg
[si:Noun] si:Noun+A3sg|ci:Agt→Noun+A3sg+m:P1sg
[gibi:Postp, PCNom] gibi:Postp
[gibi:Postp, PCGen] gibi:Postp
[Gib:Noun, Abbrv] gib:Noun+A3sg+i:Acc
[akmak:Verb] ak:Verb|an:PresPart→Adj
[aka:Noun] aka:Noun+A3sg+n:P2sg
[bir:Adj] bir:Adj
[bir:Adv] bir:Adv
[bir:Det] bir:Det
[bir:Num, Card] bir:Num
[çeşme:Noun] çeşme:Noun+A3sg
[Çeşme:Noun, Prop] çeşme:Noun+A3sg
[varmak:Verb] var:Verb+dı:Past+A3sg
[var:Adj] var:Adj|Zero→Verb+dı:Past+A3sg
[var:Noun] var:Noun+A3sg|Zero→Verb+dı:Past+A3sg
Sentence 53: ['Biraz', 'hava', 'alacağım.']
[biraz:Adj] biraz:Adj
[biraz:Adv] biraz:Adv
[hava:Adj] hava:Adj
[hav:Noun] hav:Noun+A3sg+a:Dat
[hava:Noun] hava:Noun+A3sg
[almak:Verb] al:Verb+acağ:Fut+ım:A1sg
[almak:Verb] al:Verb|acağ:FutPart→Adj+ım:P1sg
[alacak:Noun] alacağ:Noun+A3sg+ım:P1sg
[almak:Verb] al:Verb|acağ:FutPart→Noun+A3sg+ım:P1sg
[alacak:Noun] alacağ:Noun+A3sg|Zero→Verb+Pres+ım:A1sg
Sentence 54: ['İşte', 'tam', 'orada.']
[işte:Adv] işte:Adv
[iş:Noun] iş:Noun+A3sg+te:Loc
[tam

[hastalık:Noun] hastalığ:Noun+A3sg+ın:Gen
[hastalık:Noun] hastalığ:Noun+A3sg+ın:P2sg
[hasta:Adj] hasta:Adj|lığ:Ness→Noun+A3sg+ın:Gen
[hasta:Adj] hasta:Adj|lığ:Ness→Noun+A3sg+ın:P2sg
[yakın:Adv] yakın:Adv
[yakın:Adj] yakın:Adj
[yakmak:Verb] yak:Verb+Imp+ın:A2pl
[yakınmak:Verb] yakın:Verb+Imp+A2sg
[yak:Noun] yak:Noun+A3sg+ın:Gen
[yak:Noun] yak:Noun+A3sg+ın:P2sg
[yakı:Noun] yakı:Noun+A3sg+n:P2sg
[yakın:Noun] yakın:Noun+A3sg
[zaman:Noun, Time] zaman:Noun+lar:A3pl+a:Dat
[kadar:Postp, PCGen] kadar:Postp
[kadar:Postp, PCNom] kadar:Postp
[kadar:Postp, PCDat] kadar:Postp
[Kadar:Noun, Prop] kadar:Noun+A3sg
[kadar:Noun] kadar:Noun+A3sg
[tedavi:Noun] tedavi:Noun+A3sg+si:P3sg
[mümkün:Adj] mümkün:Adj
[değil:Verb] değil:Verb+Neg+miş:Narr+A3sg
[değmek:Verb] değ:Verb|il:Pass→Verb|miş:NarrPart→Adj
[değmek:Verb] değ:Verb|il:Pass→Verb+miş:Narr+A3sg
Sentence 120: ['Şimdi', 'gözlerini', 'kapat.']
[şimdi:Adv] şimdi:Adv
[şimdi:Noun, Time] şimdi:Noun+A3sg
[göz:Noun] göz:Noun+A3sg+leri:P3pl+ni:Acc
[göz:Noun] gö

[güzel:Adj] güzel:Adj
[güzel:Adv] güzel:Adv
[güzel:Noun] güzel:Noun+A3sg
[resim:Noun] resim:Noun+A3sg
[Res:Noun, Abbrv] res:Noun+A3sg|Zero→Verb+Pres+im:A1sg
Sentence 161: ['Tam', 'tahmin', 'ettiğim', 'gibi.']
[tam:Adv] tam:Adv
[tam:Adj] tam:Adj
[tam:Noun] tam:Noun+A3sg
[tahmin:Noun] tahmin:Noun+A3sg
[etmek:Verb] et:Verb|tiğ:PastPart→Adj+im:P1sg
[etmek:Verb] et:Verb|tiğ:PastPart→Noun+A3sg+im:P1sg
[gibi:Postp, PCNom] gibi:Postp
[gibi:Postp, PCGen] gibi:Postp
[Gib:Noun, Abbrv] gib:Noun+A3sg+i:Acc
Sentence 162: ['Ben', 'bir', 'kulak', 'sancısı', 'yüzünden', 'yatıyordum.']
[ben:Noun] ben:Noun+A3sg
[ben:Pron, Pers] ben:Pron+A1sg
[bir:Adj] bir:Adj
[bir:Adv] bir:Adv
[bir:Det] bir:Det
[bir:Num, Card] bir:Num
[kulak:Noun] kulak:Noun+A3sg
[sancı:Noun] sancı:Noun+A3sg+sı:P3sg
[san:Noun] san:Noun+A3sg|cı:Agt→Noun+A3sg+sı:P3sg
[yüzünden:Adv] yüzünden:Adv
[yüz:Noun] yüz:Noun+A3sg+ün:P2sg+den:Abl
[yüz:Noun] yüz:Noun+A3sg+ü:P3sg+nden:Abl
[yüz:Num, Card] yüz:Num|Zero→Noun+A3sg+ün:P2sg+den:Abl
[yüz:Num, 

...

[titremek:Verb] titr:Verb+iyor:Prog1+du:Past+A3sg
Sentence 1492: ['Bununla', 'ilgisi', 'yok.']
[bu:Pron, Demons] bu:Pron+A3sg+nunla:Ins
[bun:Noun] bun:Noun+A3sg+un:P2sg+la:Ins
[İlgi:Noun, Prop] ilgi:Noun+A3sg+si:P3sg
[ilgi:Noun] ilgi:Noun+A3sg+si:P3sg
[yok:Adj] yok:Adj
[yok:Adv] yok:Adv
[yok:Conj] yok:Conj
[yok:Adj] yok:Adj
[yok:Noun] yok:Noun+A3sg
Sentence 1493: ['Biraz', 'sonra', 'görüşürüz.']
[biraz:Adj] biraz:Adj
[biraz:Adv] biraz:Adv
[sonra:Postp, PCAbl] sonra:Postp
[sonra:Adv] sonra:Adv
[sonra:Noun, Time] sonra:Noun+A3sg
[görüşmek:Verb] görüş:Verb+ür:Aor+üz:A1pl
Sentence 1494: ['Üzgünüm,', 'özür', 'dilerim.']
[üzgün:Adj] üzgün:Adj|Zero→Verb+Pres+üm:A1sg
[üzgün:Adj] üzgün:Adj|Zero→Noun+A3sg+üm:P1sg
[üzgü:Noun] üzgü:Noun+A3sg+n:P2sg|Zero→Verb+Pres+üm:A1sg
[özür:Noun] özür:Noun+A3sg
[Özür:Noun, Prop] özür:Noun+A3sg
[dilmek:Verb] dil:Verb+er:Aor+im:A1sg
[dilemek:Verb] dile:Verb+r:Aor+im:A1sg
Sentence 1495: ['Kim', 'bilmek', 'istiyor?']
[kim:Conj] kim:Conj
[kim:Pron, Ques] kim:Pron+A3

[arka:Noun] arka:Noun+A3sg+mız:P1pl+dan:Abl
[arka:Adj] arka:Adj|Zero→Noun+A3sg+mız:P1pl+dan:Abl
[bakmak:Verb] bak:Verb+ıyor:Prog1+du:Past+A3sg
Sentence 1575: ['Bir', 'şey', 'demeyecek', 'misin?']
[bir:Adj] bir:Adj
[bir:Adv] bir:Adv
[bir:Det] bir:Det
[bir:Num, Card] bir:Num
[şey:Noun] şey:Noun+A3sg
[demek:Verb] de:Verb+me:Neg+yecek:Fut+A3sg
[demek:Verb] de:Verb+me:Neg|yecek:FutPart→Adj
[mi:Ques] mi:Ques+Pres+sin:A2sg
[mis:Noun] mis:Noun+A3sg+in:Gen
[mis:Noun] mis:Noun+A3sg+in:P2sg
[Mi:Noun, Abbrv] mi:Noun+A3sg|Zero→Verb+Pres+sin:A2sg
[mi:Noun] mi:Noun+A3sg|Zero→Verb+Pres+sin:A2sg
Sentence 1576: ['Yetti', 'artık.']
[yetmek:Verb] yet:Verb+ti:Past+A3sg
[artık:Adv] artık:Adv
[artık:Adj] artık:Adj
[artık:Noun] artık:Noun+A3sg
Sentence 1577: ['Bunu', 'hatırlıyor', 'musun?']
[bu:Pron, Demons] bu:Pron+A3sg+nu:Acc
[bun:Noun] bun:Noun+A3sg+u:Acc
[bun:Noun] bun:Noun+A3sg+u:P3sg
[hatırlamak:Verb] hatırl:Verb+ıyor:Prog1+A3sg
[mu:Ques] mu:Ques+Pres+sun:A2sg
[Mu:Noun, Abbrv] mu:Noun+A3sg|Zero→Verb+Pre

[köy:Noun] köy:Noun+A3sg|lü:With→Adj|Zero→Noun+ler:A3pl+in:Gen
[köy:Noun] köy:Noun+A3sg|lü:With→Adj|Zero→Noun+ler:A3pl+in:P2sg
[ne:Adv] ne:Adv
[ne:Interj] ne:Interj
[ne:Conj] ne:Conj
[ne:Adj] ne:Adj
[ne:Pron, Ques] ne:Pron+A3sg
[ne:Noun] ne:Noun+A3sg
[Ne:Noun, Abbrv] ne:Noun+A3sg
[yalan:Adj] yalan:Adj|cı:Agt→Noun+A3sg
[yalan:Noun] yalan:Noun+A3sg|cı:Agt→Noun+A3sg
[mahluk:Noun] mahluk:Noun+lar:A3pl
[mahluk:Noun] mahluk:Noun+A3sg|Zero→Verb+Pres+lar:A3pl
Sentence 1626: ['Şundan', 'bundan', 'konuştular.']
[şu:Pron, Demons] şu:Pron+A3sg+ndan:Abl
[şu:Adj] şu:Adj|Zero→Noun+A3sg+n:P2sg+dan:Abl
[bu:Pron, Demons] bu:Pron+A3sg+ndan:Abl
[bun:Noun] bun:Noun+A3sg+dan:Abl
[konuşmak:Verb] konuş:Verb+tu:Past+lar:A3pl
Sentence 1627: ['Artık', 'bununla', 'geçinmeye', 'çalışacaktık.']
[artık:Adv] artık:Adv
[artık:Adj] artık:Adj
[artık:Noun] artık:Noun+A3sg
[bu:Pron, Demons] bu:Pron+A3sg+nunla:Ins
[bun:Noun] bun:Noun+A3sg+un:P2sg+la:Ins
[geçinmek:Verb] geçin:Verb+me:Neg+ye:Opt+A3sg
[geçinmek:Verb] geçin:Ve

[onmak:Verb] on:Verb+Imp+un:A2pl
[o:Pron, Demons] o:Pron+A3sg+nun:Gen
[o:Pron, Pers] o:Pron+A3sg+nun:Gen
[o:Adj] o:Adj|Zero→Noun+A3sg+nun:Gen
[o:Adj] o:Adj|Zero→Noun+A3sg+n:P2sg+un:Gen
[on:Num, Card] on:Num|Zero→Noun+A3sg+un:Gen
[on:Num, Card] on:Num|Zero→Noun+A3sg+un:P2sg
[uyanmak:Verb] uyan:Verb|dığ:PastPart→Noun+A3sg+ın:P2sg+ı:Acc
[uyanmak:Verb] uyan:Verb|dığ:PastPart→Noun+A3sg+ı:P3sg+nı:Acc
[fark:Noun] fark:Noun+A3sg
[etmek:Verb] ed:Verb|en:PresPart→Adj
[E:Noun, Abbrv] e:Noun+A3sg+den:Abl
[ede:Noun] ede:Noun+A3sg+n:P2sg
[karşı:Postp, PCDat] karşı:Postp
[karşı:Adv] karşı:Adv
[karşı:Adj] karşı:Adj
[karşı:Noun] karşı:Noun+A3sg
[karşı:Noun] karşı:Noun+A3sg+Dat
[yatak:Noun] yatak:Noun+A3sg+ta:Loc|ki:Rel→Adj
[sakal:Noun] sakal:Noun+A3sg+ın:P2sg+ı:Acc
[sakal:Noun] sakal:Noun+A3sg+ı:P3sg+nı:Acc
[saka:Noun] saka:Noun+A3sg|lı:With→Adj|Zero→Noun+A3sg+n:P2sg+ı:Acc
[sıvazlamak:Verb] sıvazla:Verb|yarak:ByDoingSo→Adv
Sentence 1690: ['Öyle', 'bir', 'uyku', 'ki,', 'ne', 'çarpıntısı', 'vardır,', 'ne

AttributeError: 'NoneType' object has no attribute 'content'

In [19]:
k = 3
index = df["sentence"][k].split()
print(f"Sentence {k+1}: {index}")
    
for k in index:
    k = k.strip(string.punctuation)
    cv_analysis = morphology.analyze(k)
    for cv_result in cv_analysis:
        print(cv_result)

Sentence 4: ['Ne', 'dersiniz?']
[ne:Adv] ne:Adv
[ne:Interj] ne:Interj
[ne:Conj] ne:Conj
[ne:Adj] ne:Adj
[ne:Pron, Ques] ne:Pron+A3sg
[ne:Noun] ne:Noun+A3sg
[Ne:Noun, Abbrv] ne:Noun+A3sg
[demek:Verb] de:Verb+r:Aor+siniz:A2pl
[ders:Noun] ders:Noun+A3sg+iniz:P2pl
[ders:Noun] ders:Noun+A3sg+in:Gen|Zero→Verb+Pres+iz:A1pl
[ders:Noun] ders:Noun+A3sg+in:P2sg|Zero→Verb+Pres+iz:A1pl
